In [2]:
import numpy as np
import string
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import time

In [3]:
st = time.time()

In [4]:
def preprocess(text):
    
    preprocessed_text = text.lower().replace('-', ' ')
    translation_table = str.maketrans('\n', ' ', string.punctuation+string.digits)
    preprocessed_text = text.translate(translation_table)
    
    return preprocessed_text

In [4]:
# df_train["lang"].unique()

In [5]:
df_3 = pd.read_csv('../../data/qald-3-train.csv')
df_4 = pd.read_csv('../../data/qald-4-train.csv')
df_5 = pd.read_csv('../../data/qald-5-train.csv')
df_6 = pd.read_csv('../../data/qald-6-train.csv')
df_7 = pd.read_csv('../../data/qald-7-train.csv')
df_8 = pd.read_csv('../../data/qald-8-train.csv')
df_9 = pd.read_csv('../../data/qald-9-train.csv')

In [6]:
a = df_9[df_9.lang=="ru"].head()

In [7]:
a['questions'] = [preprocess(text) for text in a['questions'].values]

In [8]:
a.head()

,questions,keywords,lang
1,Список все настольные игры от ВРЕМЯ ПО ГРИНВИЧУ,"настольная игра , время по Гринвичу",ru
12,Кто развитая Skype,"развивать , Skype",ru
23,Который люди мы Родился в Ираклион,"люди , Родился , ираклион",ru
34,В который УС государство является Площадь рас...,"Площадь 51 , располагается , У.С. государство",ru
45,Кто является мэр из новый Йорк Город,"новый Йорк город , мэр",ru


In [8]:
a.questions.dtype

dtype('O')

In [9]:
frames = [df_3,df_4,df_5,df_6,df_7,df_8,df_9 ]

df_train = pd.concat(frames)
df_train=df_train.dropna(subset=['questions'])

In [10]:
df_train["lang"].unique()

array(['en', 'de', 'es', 'it', 'fr', 'nl', 'ro', 'fa', 'pt_BR', 'hi_IN',
       'ru', 'pt'], dtype=object)

In [11]:
vectorizer = CountVectorizer()

In [12]:
df_train['questions'] = [preprocess(text) for text in df_train['questions'].values]

In [13]:
X_train = vectorizer.fit_transform(df_train["questions"])
y_train = df_train["lang"]

In [14]:
naive_classifier = MultinomialNB(fit_prior=False,alpha=0.01)
naive_classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.01, fit_prior=False)

In [15]:
end = time.time()

In [16]:
t = end-st

In [19]:
# l = ["../data/en_entity_label.csv","../data/de_entity_label.csv","../data/it_entity_label.csv","../data/es_entity_label.csv","../data/fr_entity_label.csv","../data/pt_entity_label.csv","../data/ru_entity_label.csv"]

l = ["../data/entity_labels/it_entity_label.csv","../data/entity_labels/fr_entity_label.csv","../data/entity_labels/pt_entity_label.csv","../data/entity_labels/ru_entity_label.csv"]

scores = []
times = []

for d in l:
    df_test = pd.read_csv("../"+d)
    df_test=df_test.dropna(subset=['questions'])
    df_test['questions'] = [preprocess(text) for text in df_test['questions'].values]
    
    X_test = vectorizer.transform(df_test["questions"])
    y_test = df_test["lang"]
    
    a = time.time()
    predictions = naive_classifier.predict(X_test)
    b = time.time()
    c = b-a + t
    
    times.append(c/df_test.shape[0])
    
    scores.append(accuracy_score(y_test, predictions))

In [20]:
scores

[0.3607843137254902,
 0.3742889647326507,
 0.1774193548387097,
 0.13636363636363635]

In [20]:
times

[6.907408995479533e-05,
 7.532246170205686e-05,
 0.002484229366475172,
 7.579885801836364e-05,
 0.0007719950907675506,
 0.0026593176803924963,
 0.007272660014141037]

In [21]:
df = pd.read_csv("../../data/qald-9-test.csv")

In [22]:
df.lang.unique()

array(['de', 'ru', 'pt', 'en', 'hi_IN', 'fa', 'pt_BR', 'it', 'fr', 'ro',
       'es', 'nl'], dtype=object)

In [23]:
a = time.time()
df_test = pd.read_csv("../../data/pt_entity_label.csv")
# 
df_test=df_test.dropna(subset=['questions'])
df_test['questions'] = [preprocess(text) for text in df_test['questions'].values]

X_test = vectorizer.transform(df_test["questions"])
y_test = ["pt"]*df_test.shape[0]

#     a = datetime.datetime.now()

predictions = naive_classifier.predict(X_test)
#     b = datetime.datetime.now()
b = time.time()
c = b-a+t

#     print(df_test.shape)

times.append(c/df_test.shape[0])

scores.append(accuracy_score(y_test, predictions))

In [24]:
times

[6.907408995479533e-05,
 7.532246170205686e-05,
 0.002484229366475172,
 7.579885801836364e-05,
 0.0007719950907675506,
 0.0026593176803924963,
 0.007272660014141037,
 0.0027299452458184196]

In [25]:
scores

[0.2635189207728364,
 0.8822795859832897,
 0.35802469135802467,
 0.15372066758689923,
 0.3631713554987212,
 0.18061674008810572,
 0.12048192771084337,
 0.18061674008810572]

In [26]:
scores[-1]+scores[-2]

0.3010986677989491

In [27]:
df_test = pd.read_csv("../../data/ru_entity_label.csv")
df_test=df_test.dropna(subset=['questions'])
df_test['questions'] = [preprocess(text) for text in df_test['questions'].values]

# df_test['questions'] = [text.encode("utf-8") for text in df_test['questions'].values]

X_test = vectorizer.transform(df_test["questions"])
y_test = df_test["lang"]

a = time.time()
predictions = naive_classifier.predict(X_test)
b = time.time()
c = b-a + t

times.append(c/df_test.shape[0])

scores.append(accuracy_score(y_test, predictions))

In [28]:
times

[6.907408995479533e-05,
 7.532246170205686e-05,
 0.002484229366475172,
 7.579885801836364e-05,
 0.0007719950907675506,
 0.0026593176803924963,
 0.007272660014141037,
 0.0027299452458184196,
 0.007277569138860128]

In [29]:
scores

[0.2635189207728364,
 0.8822795859832897,
 0.35802469135802467,
 0.15372066758689923,
 0.3631713554987212,
 0.18061674008810572,
 0.12048192771084337,
 0.18061674008810572,
 0.12048192771084337]